In [1]:
using CLOUD.ConservationLaws
using CLOUD.SpatialDiscretizations
using CLOUD.Solvers
using CLOUD.InitialConditions
using CLOUD.Mesh
using CLOUD.Analysis
using CLOUD.IO

using OrdinaryDiffEq
using TimerOutputs

"Set up paths"
plots_path = "../plots/advection1d_demo/"
results_path = "../results/advection1d_demo/"

"Define the physical problem"
a = 1.0  # advection velocity
A = 1.0  # amplitude
L = 1.0  # domain length
k = 2*π/L  # wave number
T = 1*L/a  # end time

# constant-coefficient linear adection equation with sine wave initial condition
initial_data = InitialDataSine(A,k)
conservation_law = linear_advection_equation(a)
u_exact = initial_condition(initial_data, conservation_law)

"Set discretization parameters"
M = 100  # number of elements
p = 2  # degree of discretization
N_q = 3  # number of quadrature points
reference_approximation=ReferenceApproximation(DGSEM(p), 
    Line(), LGQuadrature())
form = StrongConservationForm()
strategy = Eager()
dt_scale=0.1

"Generate a uniform periodic mesh"
mesh = uniform_periodic_mesh(reference_approximation.reference_element, 
    (0.0,L), M)

"Construct spatial discretization"
spatial_discretization = SpatialDiscretization(mesh, reference_approximation)

ode_problem = semidiscretize(conservation_law, 
    spatial_discretization,
    initial_data, form,
    (0.0, T), strategy);

┌ Info: Precompiling CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2]
└ @ Base loading.jl:1342
ERROR: LoadError: LoadError: invalid using path: "IO" does not name a module
Stacktrace:
 [1] include(mod::Module, _path::String)
   @ Base ./Base.jl:386
 [2] include(x::String)
   @ CLOUD ~/Research/CLOUD.jl/src/CLOUD.jl:1
 [3] top-level scope
   @ ~/Research/CLOUD.jl/src/CLOUD.jl:12
 [4] include
   @ ./Base.jl:386 [inlined]
 [5] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::Nothing)
   @ Base ./loading.jl:1235
 [6] top-level scope
   @ none:1
 [7] eval
   @ ./boot.jl:360 [inlined]
 [8] eval(x::Expr)
   @ Base.MainInclude ./client.jl:446
 [9] top-level scope
   @ none:1
in expression starting at /Users/tristanmontoya/Research/CLOUD.jl/src/Analysis/Analysis.jl:1
in expression starting at /Users/tristanmontoya/Research/CLOUD.jl/src/CLOUD.jl:1


LoadError: Failed to precompile CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2] to /Users/tristanmontoya/.julia/compiled/v1.6/CLOUD/jl_uE7S0T.

In [ ]:
"run the solver"
write_interval=100
sol = solve(ode_problem, RK4(), adaptive=false, 
    dt=dt_scale*(L/M)/(a*(2*p+1)), save_everystep=false,
    callback=save_solution(results_path, write_interval))
print_timer()
reset_timer!();

In [ ]:
"visualize the solution"
plotter = Plotter(spatial_discretization, plots_path)
error_analysis = ErrorAnalysis(spatial_discretization, results_path)
visualize(last(sol.u), plotter,
    "final.pdf", exact_solution=u_exact,
     label="U^h(x,t)", label_exact="U(x,t)")
print("L2 error: ", analyze(error_analysis, last(sol.u), u_exact))